In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
from memory_profiler import memory_usage

fact_calls = 0
def factorial(n):
    global fact_calls
    fact_calls += 1
    if n <= 1:
        return 1
    return n * factorial(n - 1)

fib_calls = 0
def fib_naive(n):
    global fib_calls
    fib_calls += 1
    if n <= 1:
        return n
    return fib_naive(n - 1) + fib_naive(n - 2)

fib_dp_calls = 0
def fib_dp(n, memo):
    global fib_dp_calls
    fib_dp_calls += 1
    if n in memo:
        return memo[n]
    memo[n] = fib_dp(n - 1, memo) + fib_dp(n - 2, memo)
    return memo[n]

def fact_wrap(n): factorial(n)
def fib_wrap(n): fib_naive(n)
def fib_dp_wrap(n): fib_dp(n, {0: 0, 1: 1})

inputs = [5, 10, 15, 20]
result = []

fact_t, fib_t, fibdp_t = [], [], []
fact_m, fib_m, fibdp_m = [], [], []

for n in inputs:
    fact_calls = 0
    s = time.perf_counter()
    factorial(n)
    fact_time = time.perf_counter() - s
    fact_mem = max(memory_usage((fact_wrap, (n,)))) - min(memory_usage((fact_wrap, (n,))))

    fib_calls = 0
    s = time.perf_counter()
    fib_naive(n)
    fib_time = time.perf_counter() - s
    fib_mem = max(memory_usage((fib_wrap, (n,)))) - min(memory_usage((fib_wrap, (n,))))

    fib_dp_calls = 0
    s = time.perf_counter()
    fib_dp(n, {0: 0, 1: 1})
    fib_dp_time = time.perf_counter() - s
    fib_dp_mem = max(memory_usage((fib_dp_wrap, (n,)))) - min(memory_usage((fib_dp_wrap, (n,))))

    fact_t.append(fact_time)
    fib_t.append(fib_time)
    fibdp_t.append(fib_dp_time)

    fact_m.append(fact_mem)
    fib_m.append(fib_mem)
    fibdp_m.append(fib_dp_mem)

    result.append([
        n,
        fact_time, fact_calls,
        fib_time, fib_calls,
        fib_dp_time, fib_dp_calls
    ])

df = pd.DataFrame(
    result,
    columns=[
        "n",
        "Factorial Time", "Factorial Calls",
        "Fibonacci Naive Time", "Fibonacci Naive Calls",
        "Fibonacci DP Time", "Fibonacci DP Calls"
    ]
)

print(df)

plt.figure()
plt.plot(inputs, fact_t, marker='o', label='Factorial')
plt.plot(inputs, fib_t, marker='o', label='Fibonacci Naive')
plt.plot(inputs, fibdp_t, marker='o', label='Fibonacci DP')
plt.xlabel("n")
plt.ylabel("Time (seconds)")
plt.title("Time Complexity Comparison")
plt.legend()
plt.show()

plt.figure()
plt.plot(inputs, fact_m, marker='o', label='Factorial')
plt.plot(inputs, fib_m, marker='o', label='Fibonacci Naive')
plt.plot(inputs, fibdp_m, marker='o', label='Fibonacci DP')
plt.xlabel("n")
plt.ylabel("Memory Used (MiB)")
plt.title("Space Complexity using Memory Profiler")
plt.legend()
plt.show()
